In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
import numpy.random as npr

In [289]:
S_a = 52
S_b = 65
X_1 = 50
X_2 = 70
rf = 0.1
b_1 = 0.1
b_2 = 0.1
sigma_1 = 0.2
sigma_2 = 0.3
rho = 0.75
T = 0.5

y1 = (np.log(S_a / X_1) + (rf - b_1 - sigma_1 ** 2 / 2) * T) / (sigma_1 * np.sqrt(T))
y2 = (np.log(S_b / X_2) + (rf - b_2 - sigma_2 ** 2 / 2) * T) / (sigma_2 * np.sqrt(T))

In [295]:
def std_bi_nd_cdf(x, y, rho):
    return sps.multivariate_normal.cdf([x, y], mean=[0, 0], cov=np.array([[1, rho], [rho, 1]]))

In [296]:
m1 = std_bi_nd_cdf(y2 + sigma_2 * np.sqrt(T), y1 + rho * sigma_2 * np.sqrt(T), rho)
m2 = std_bi_nd_cdf(y2, y1, rho)

In [298]:
c = S_b * np.exp((-b_2 - rf*0) * T) * m1 - X_2 * np.exp(- rf * T) * m2
c

3.242527897551966

In [232]:
corr = np.array([[1, rho], [rho, 1]])
print(f"Input correlation matrix: \n{corr}\n")

chol = np.linalg.cholesky(corr)
z = np.matmul(chol, npr.normal(size=(2, 10000000)))

Input correlation matrix: 
[[1.   0.75]
 [0.75 1.  ]]



In [233]:
# The GBM formula
def gbm(S_0, rf, q, sigma, delta_t, z):
    return S_0 * np.exp((rf - q - sigma**2 / 2) * delta_t + sigma * np.sqrt(delta_t) * z)

In [234]:
p1 = gbm(S_a, rf, b_1, sigma_1, T, z[0, :])
p2 = gbm(S_b, rf, b_2, sigma_2, T, z[1, :])

# Calculate the correlation between the two price paths' return
print(np.corrcoef(p1 / S_a - 1, p2 / S_b - 1))

[[1.         0.74628276]
 [0.74628276 1.        ]]


In [235]:
callpayoff = np.maximum(p2 - X_2, 0) * (p1 > X_1)

In [275]:
np.exp(- rf * T) * callpayoff.mean()

3.240683509312446

In [252]:
def mc_int_2d(func, rg1, rg2, N=int(1e6)):
    nr1 = npr.uniform(rg1[0], rg1[1], size=N)
    nr2 = npr.uniform(rg2[0], rg2[1], size=N)
    sums = func(nr1, nr2)
    print(sums.shape)
    res = np.mean(sums) * (rg1[1] - rg1[0]) * (rg2[1] - rg2[0])
    std = np.std(sums) * (rg1[1] - rg1[0]) * (rg2[1] - rg2[0]) / np.sqrt(N)
    return res, std

In [267]:
def call_payoff(S1, S2, K1, K2):
    return np.maximum(S2 - K2, 0) * (S1 > K1)


# def put_payoff(S, strike):
#     return np.maximum(strike - S, 0)

def option_payoff(S_a, S_b, K1, K2, T, rf, sigma_1, sigma_2, payoff_func, z1, z2):
    ST_a = gbm(S_a, rf, b_1, sigma_1, T, z1)
    ST_b = gbm(S_b, rf, b_2, sigma_2, T, z2)
    return payoff_func(ST_a, ST_b, K1, K2)

In [268]:
def rainbow_opt_integrand(S_a, S_b, K1, K2, T, rf, sigma_1, sigma_2, rho, payoff_func):
    def _inner(z1, z2, payoff_func=payoff_func):
        return option_payoff(
            S_a, S_b, K1, K2, T, rf, sigma_1, sigma_2, payoff_func, z1, z2
        ) * sps.multivariate_normal.pdf(np.vstack((z1, z2)).T, mean=[0, 0], cov=np.array([[1, rho], [rho, 1]]))

    return _inner

In [274]:
np.exp(-rf * T) * mc_int_2d(rainbow_opt_integrand(S_a, S_b, X_1, X_2, T, rf, sigma_1, sigma_2, rho, call_payoff), (-10, 10), (-10, 10), int(10000000))[0]

(10000000,)


3.248736296038302